# Polar Coding Notebook

In [1]:
# FINAL MINIMAL SELF-CONTAINED DIAGNOSTIC
# Restart kernel, paste this single cell and run it.

import numpy as np
from math import log2

# ---- definitions (all fresh) ----
def polar_transform(u):
    N = len(u)
    if N == 1:
        return u.copy()
    u_even = u[0:N:2]
    u_odd  = u[1:N:2]
    upper = polar_transform((u_even ^ u_odd) % 2)
    lower = polar_transform(u_odd)
    return np.concatenate([upper, lower]).astype(np.int8)

def kron_F_power(N):
    assert (N & (N-1)) == 0 and N>0
    F = np.array([[1,0],[1,1]], dtype=int)
    n = int(log2(N))
    G = np.array([[1]], dtype=int)
    for _ in range(n):
        G = np.kron(G, F)
    return G % 2

def f_exact(a,b,eps=1e-12):
    A = np.array(a, dtype=float); B = np.array(b, dtype=float)
    ta = np.tanh(A/2.0); tb = np.tanh(B/2.0)
    t = np.clip(ta*tb, -1+eps, 1-eps)
    return np.log1p(t) - np.log1p(-t)  # = 2*atanh(t)

def sc_decode_exact_instrumented(llr, frozen, u_frozen=None):
    import numpy as _np
    if u_frozen is None:
        u_frozen = _np.zeros(len(frozen), dtype=_np.int8)
    llr = _np.array(llr, dtype=float)
    frozen = _np.array(frozen, dtype=bool)
    u_frozen = _np.array(u_frozen, dtype=_np.int8)
    def rec(llr_sub, frozen_sub, u_frozen_sub, depth=0, path="root"):
        indent = "  "*depth
        n = len(llr_sub)
        print(f"{indent}{path}: n={n}, llr_sub={llr_sub.tolist()}, frozen_sub={frozen_sub.tolist()}")
        if n==1:
            if frozen_sub[0]:
                print(f"{indent} => leaf frozen -> {u_frozen_sub[0]}")
                return _np.array([u_frozen_sub[0]],dtype=_np.int8)
            dec = 0 if llr_sub[0] >= 0 else 1
            print(f"{indent} => leaf decide -> {dec}")
            return _np.array([dec], dtype=_np.int8)
        n2 = n//2
        a = llr_sub[:n2]; b = llr_sub[n2:]
        f = f_exact(a,b)
        print(f"{indent} f: a={a.tolist()}, b={b.tolist()} -> f={f.tolist()}")
        u_up = rec(f, frozen_sub[:n2], u_frozen_sub[:n2], depth+1, path=path+"-L")
        print(f"{indent} u_up returned: {u_up.tolist()}")
        sign = (1-2*u_up)
        g = b + sign * a
        print(f"{indent} g: sign={sign.tolist()}, g={g.tolist()}")
        u_low = rec(g, frozen_sub[n2:], u_frozen_sub[n2:], depth+1, path=path+"-R")
        print(f"{indent} u_low returned: {u_low.tolist()}")
        combined = _np.concatenate([(u_up ^ u_low).astype(_np.int8), u_low]).astype(_np.int8)
        print(f"{indent} combined: {combined.tolist()}")
        return combined
    return rec(llr, frozen, u_frozen, depth=0, path="root")

# ---- test parameters ----
N = 4
K = N//2
p_design = 0.05

# compute logical reliabilities and map to natural-order positions
def bhattacharyya(p,N):
    Z = np.array([2.0*np.sqrt(p*(1-p))])
    while len(Z)<N:
        Zn=[]
        for z in Z:
            Zn.append(2*z - z*z); Zn.append(z*z)
        Z=np.array(Zn)
    return Z

Z = bhattacharyya(p_design, N)
logical_best = np.argsort(Z)[:K]
br = np.array([int(format(i,'b').zfill(int(log2(N)))[::-1],2) for i in range(N)])
info_nat = np.sort(br[logical_best])

# build u (natural order) with deterministic message
msg = np.array([0,1], dtype=np.int8)
u = np.zeros(N, dtype=np.int8); u[info_nat] = msg
x = polar_transform(u)
Gnat = kron_F_power(N)
x_g = (u.astype(int) @ Gnat.astype(int)) % 2

print("N,K:",N,K)
print("logical_best:", logical_best.tolist())
print("bit-reverse br:", br.tolist())
print("info positions (natural):", info_nat.tolist())
print("u natural:", u.tolist())
print("x (transform):", x.tolist())
print("x (u@Gnat):", x_g.tolist(), "match:", np.array_equal(x,x_g))

# generate noiseless LLRs
eps = 1e-12
alpha = np.log((1.0-eps)/eps)
y = x.copy()
llr = (1-2*y) * alpha
print("llr:", llr.tolist())
print("frozen_mask (natural order):", (~np.isin(np.arange(N), info_nat)).tolist())

print("\nRun instrumented exact-SC now:")
uhat = sc_decode_exact_instrumented(llr, (~np.isin(np.arange(N), info_nat)), u_frozen=np.zeros(N,dtype=np.int8))
print("\nfinal uhat:", uhat.tolist())
print("expected u:", u.tolist())
print("match?", np.array_equal(uhat, u))


N,K: 4 2
logical_best: [3, 2]
bit-reverse br: [0, 2, 1, 3]
info positions (natural): [1, 3]
u natural: [0, 0, 0, 1]
x (transform): [1, 1, 1, 1]
x (u@Gnat): [1, 1, 1, 1] match: True
llr: [-27.63102111592755, -27.63102111592755, -27.63102111592755, -27.63102111592755]
frozen_mask (natural order): [True, False, True, False]

Run instrumented exact-SC now:
root: n=4, llr_sub=[-27.63102111592755, -27.63102111592755, -27.63102111592755, -27.63102111592755], frozen_sub=[True, False, True, False]
 f: a=[-27.63102111592755, -27.63102111592755], b=[-27.63102111592755, -27.63102111592755] -> f=[26.937840546492907, 26.937840546492907]
  root-L: n=2, llr_sub=[26.937840546492907, 26.937840546492907], frozen_sub=[True, False]
   f: a=[26.937840546492907], b=[26.937840546492907] -> f=[26.244693365930964]
    root-L-L: n=1, llr_sub=[26.244693365930964], frozen_sub=[True]
     => leaf frozen -> 0
   u_up returned: [0]
   g: sign=[1], g=[53.875681092985815]
    root-L-R: n=1, llr_sub=[53.875681092985815]

In [2]:
# Patched Polar Coding implementation (natural-order encoder + SC decoder)
# Paste this file into a fresh Jupyter cell or save as polar_working.py
# Restart the kernel before running to avoid stale definitions.

import numpy as np
import matplotlib.pyplot as plt

# ----------------------------
# Utilities
# ----------------------------

def is_power_of_two(n):
    return (n & (n-1)) == 0 and n > 0

def bit_reverse_indices(N):
    n = int(np.log2(N))
    br = np.zeros(N, dtype=int)
    for i in range(N):
        br[i] = int(format(i, 'b').zfill(n)[::-1], 2)
    return br

# ----------------------------
# Recursive polar transform (natural-order)
# ----------------------------

def polar_transform(u):
    """Recursive natural-order polar transform: returns x = polar_transform(u)."""
    N = len(u)
    if N == 1:
        return u.copy()
    u_even = u[0:N:2]
    u_odd  = u[1:N:2]
    upper = polar_transform((u_even ^ u_odd) % 2)
    lower = polar_transform(u_odd)
    return np.concatenate([upper, lower]).astype(np.int8)


def polar_encode_recursive(msg, info_positions, N, u_frozen=None):
    """Place info bits at info_positions (natural-order), frozen bits per u_frozen, return x,u."""
    if u_frozen is None:
        u_frozen = np.zeros(N, dtype=np.int8)
    u = np.array(u_frozen, dtype=np.int8)
    u[np.sort(info_positions)] = np.array(msg, dtype=np.int8)
    x = polar_transform(u)
    return x.astype(np.int8), u.astype(np.int8)

# ----------------------------
# Bhattacharyya-based frozen selection (logical -> natural mapping)
# ----------------------------

def bhattacharyya_parameter(p, N):
    """Compute vector Z of length N by recursion (starting from channel with param p)."""
    Z = np.array([2.0 * np.sqrt(p * (1.0 - p))])
    while len(Z) < N:
        Z_next = []
        for z in Z:
            Z_next.append(2*z - z*z)  # upper
            Z_next.append(z*z)        # lower
        Z = np.array(Z_next)
    return Z


def select_frozen_bits_logical_to_natural(N, K, p_design):
    """
    Select K best logical channels (using Bhattacharyya Z),
    then map those logical indices to natural-order positions via bit-reversal:
      natural_positions = br[logical_best]
    This is a standard method to get info positions for the recursive polar_transform.
    """
    Z = bhattacharyya_parameter(p_design, N)
    logical_best = np.argsort(Z)[:K]   # best logical indices (0..N-1)
    br = bit_reverse_indices(N)
    info_positions_natural = np.sort(br[logical_best])
    frozen_mask = np.ones(N, dtype=bool)
    frozen_mask[info_positions_natural] = False
    return frozen_mask, info_positions_natural

# ----------------------------
# Channel models and LLR
# ----------------------------

def bsc(x, p):
    flips = (np.random.rand(len(x)) < p).astype(np.int8)
    return (x ^ flips).astype(np.int8)


def bsc_llr(y, p, eps=1e-12):
    p_safe = min(max(float(p), eps), 1.0 - eps)
    alpha = np.log((1.0 - p_safe) / p_safe)
    return (1 - 2*y) * alpha

# ----------------------------
# LLR combining helpers
# ----------------------------

def f_min_sum(a, b):
    a = np.array(a); b = np.array(b)
    return np.sign(a * b) * np.minimum(np.abs(a), np.abs(b))


def f_exact(a, b, eps=1e-12):
    A = np.array(a, dtype=float)
    B = np.array(b, dtype=float)
    ta = np.tanh(A / 2.0)
    tb = np.tanh(B / 2.0)
    t = ta * tb
    t = np.clip(t, -1.0 + eps, 1.0 - eps)
    return np.log1p(t) - np.log1p(-t)  # = 2*atanh(t)

# ----------------------------
# Natural-order SC decoder
# ----------------------------

def successive_cancellation_decode(llr, frozen_mask, u_frozen=None, use_exact_f=False):
    import numpy as _np
    if u_frozen is None:
        u_frozen = _np.zeros(len(frozen_mask), dtype=_np.int8)
    N = len(llr)
    llr = _np.array(llr, dtype=float)
    frozen_mask = _np.array(frozen_mask, dtype=bool)
    u_frozen = _np.array(u_frozen, dtype=_np.int8)

    def sc_rec(llr_sub, frozen_sub, u_frozen_sub):
        n = len(llr_sub)
        if n == 1:
            if frozen_sub[0]:
                return _np.array([u_frozen_sub[0]], dtype=_np.int8)
            else:
                return _np.array([0 if llr_sub[0] >= 0 else 1], dtype=_np.int8)
        n2 = n // 2
        a = llr_sub[:n2]
        b = llr_sub[n2:]
        f_llr = f_exact(a, b) if use_exact_f else f_min_sum(a, b)
        uhat_upper = sc_rec(f_llr, frozen_sub[:n2], u_frozen_sub[:n2])
        sign_term = (1 - 2 * uhat_upper)
        g_llr = b + sign_term * a
        uhat_lower = sc_rec(g_llr, frozen_sub[n2:], u_frozen_sub[n2:])
        u_upper = (uhat_upper ^ uhat_lower).astype(_np.int8)
        return _np.concatenate([u_upper, uhat_lower]).astype(_np.int8)

    return sc_rec(llr, frozen_mask, u_frozen)

# ----------------------------
# Monte-Carlo driver (natural-order pairing)
# ----------------------------

def monte_carlo_polar(N=128, K=None, p_design=0.05, p_channel=None,
                      trials=1000, use_exact_f=False, verbose=False):
    """
    Robust Monte-Carlo for polar codes using natural-order encoder/decoder.
    Auto-detects whether logical->natural mapping should use br or inv_br.
    """
    if not is_power_of_two(N):
        raise ValueError("N must be power of two")
    if K is None:
        K = N // 2
    if p_channel is None:
        p_channel = p_design

    # 1) compute logical best channels
    Z = bhattacharyya_parameter(p_design, N)
    logical_best = np.argsort(Z)[:K]

    # 2) prepare candidate mappings
    br = bit_reverse_indices(N)
    inv_br = np.empty_like(br)
    inv_br[br] = np.arange(N)

    candidate_map_names = ['br', 'inv_br']
    candidate_info_positions = [
        np.sort(br[logical_best]),
        np.sort(inv_br[logical_best])
    ]

    # 3) auto-detect which mapping yields correct decode for a small deterministic test
    chosen_info_positions = candidate_info_positions[0]  # default
    chosen_map_name = candidate_map_names[0]
    # deterministic small test message & frozen values
    test_msg = None
    # build u_frozen zeros
    u_frozen_test = np.zeros(N, dtype=np.int8)
    # We'll try each candidate mapping by performing a single noiseless encode->decode test
    for name, info_pos in zip(candidate_map_names, candidate_info_positions):
        # construct frozen mask for test
        frozen_mask_test = np.ones(N, dtype=bool)
        frozen_mask_test[info_pos] = False
        # deterministic message: 0,1,2.. mod 2 to fill K bits
        km = len(info_pos)
        test_msg = np.arange(km, dtype=np.int8) % 2
        x_test, u_test = polar_encode_recursive(test_msg, info_pos, N, u_frozen=u_frozen_test)
        # noiseless channel llr
        y_test = x_test.copy()
        llr_test = bsc_llr(y_test, 1e-12)  # very small eps => very high magnitude LLRs
        # decode using exact f for robustness in the test
        uhat_test = successive_cancellation_decode(llr_test, frozen_mask_test, u_frozen=u_frozen_test, use_exact_f=True)
        if np.array_equal(uhat_test, u_test):
            chosen_info_positions = info_pos
            chosen_map_name = name
            break
    else:
        # Neither mapping passed the deterministic test — fall back to br mapping but warn
        chosen_info_positions = candidate_info_positions[0]
        chosen_map_name = candidate_map_names[0]
        print("Warning: auto-detect did not find a mapping that passes the deterministic no-noise test. "
              "Falling back to 'br' mapping. You can run instrumented_n4_check() to inspect.")

    # Print which mapping was selected (helpful)
    if verbose:
        print(f"Selected mapping: {chosen_map_name}, info_positions (natural) = {chosen_info_positions.tolist()}")

    # Prepare frozen mask & u_frozen for the full simulation
    frozen_mask, info_positions = np.ones(N, dtype=bool), chosen_info_positions
    frozen_mask[info_positions] = False
    u_frozen = np.zeros(N, dtype=np.int8)
    info_positions = np.sort(info_positions)
    K_actual = len(info_positions)

    # 4) Run Monte-Carlo trials
    bit_err = 0
    block_err = 0
    for t in range(trials):
        msg = np.random.randint(0,2,size=K_actual).astype(np.int8)
        x, u = polar_encode_recursive(msg, info_positions, N, u_frozen=u_frozen)
        y = bsc(x, p_channel)
        llr = bsc_llr(y, p_channel)
        uhat = successive_cancellation_decode(llr, frozen_mask, u_frozen=u_frozen, use_exact_f=use_exact_f)
        msg_hat = uhat[info_positions]
        this_err = int(np.sum(msg_hat != msg))
        bit_err += this_err
        block_err += int(this_err > 0)
        if verbose and (t % max(1, trials//10) == 0):
            print(f"Trial {t}/{trials}: bit_err={bit_err}, block_err={block_err}")
    ber = bit_err / (trials * K_actual)
    bler = block_err / trials
    # Also return which mapping was chosen for debugging
    return ber, bler, chosen_map_name, info_positions


# ----------------------------
# Quick sanity tests and helpers
# ----------------------------

def quick_sanity_tests():
    print("=== Quick sanity tests ===")
    for N in (4, 8, 16, 32):
        K = N // 2
        print(f"\nTesting N={N}, K={K} (design p=0.01)...")
        ber0, bler0 = monte_carlo_polar(N=N, K=K, p_design=0.01, p_channel=0.0, trials=200, use_exact_f=True)
        print("No-noise check: BER =", ber0, "BLER =", bler0)
        ber5, bler5 = monte_carlo_polar(N=N, K=K, p_design=0.5, p_channel=0.5, trials=200, use_exact_f=True)
        print("p=0.5 check : BER =", ber5, "BLER =", bler5)


def instrumented_n4_check(p_design=0.05, p_channel=0.0, use_exact_f=True):
    N = 4; K = 2
    frozen_mask, info_positions = select_frozen_bits_logical_to_natural(N, K, p_design)
    u_frozen = np.zeros(N, dtype=np.int8)
    msg = np.array([0,1], dtype=np.int8)
    x, u = polar_encode_recursive(msg, info_positions, N, u_frozen=u_frozen)
    print("info_positions (natural):", info_positions.tolist())
    print("u (natural):", u.tolist())
    print("x:", x.tolist())
    y = bsc(x, p_channel)
    llr = bsc_llr(y, p_channel)
    print("llr:", llr.tolist())
    def sc_rec_print(llr_sub, frozen_sub, u_frozen_sub, depth=0, path="root"):
        indent = "  "*depth
        n = len(llr_sub)
        print(f"{indent}{path}: n={n}, llr_sub={llr_sub.tolist()}, frozen_sub={frozen_sub.tolist()}")
        if n==1:
            if frozen_sub[0]:
                print(f"{indent} => leaf frozen -> {u_frozen_sub[0]}")
                return np.array([u_frozen_sub[0]], dtype=np.int8)
            dec = 0 if llr_sub[0] >= 0 else 1
            print(f"{indent} => leaf decide -> {dec}")
            return np.array([dec], dtype=np.int8)
        n2 = n//2
        a = llr_sub[:n2]; b = llr_sub[n2:]
        f_llr = f_exact(a,b) if use_exact_f else f_min_sum(a,b)
        print(f"{indent} f: a={a.tolist()}, b={b.tolist()} -> f={f_llr.tolist()}")
        uhat_up = sc_rec_print(f_llr, frozen_sub[:n2], u_frozen_sub[:n2], depth+1, path=path+"-L")
        print(f"{indent} uhat_up returned: {uhat_up.tolist()}")
        sign_term = (1 - 2 * uhat_up)
        g_llr = b + sign_term * a
        print(f"{indent} g: sign_term={sign_term.tolist()}, g_llr={g_llr.tolist()}")
        uhat_low = sc_rec_print(g_llr, frozen_sub[n2:], u_frozen_sub[n2:], depth+1, path=path+"-R")
        print(f"{indent} uhat_low returned: {uhat_low.tolist()}")
        combined = np.concatenate([(uhat_up ^ uhat_low).astype(np.int8), uhat_low]).astype(np.int8)
        print(f"{indent} combined: {combined.tolist()}")
        return combined
    uhat = sc_rec_print(llr, frozen_mask, u_frozen)
    print("decoded uhat:", uhat.tolist())
    print("expected u:", u.tolist())
    print("match?", np.array_equal(uhat, u))

# End of patched module


In [3]:
# after replacing function and restarting kernel
ber0, bler0, mapping, info_pos = monte_carlo_polar(N=32, K=16, p_design=0.01, p_channel=0.0, trials=200, use_exact_f=True, verbose=True)
print("No-noise check:", ber0, bler0, "mapping:", mapping, "info_pos:", info_pos)


Selected mapping: br, info_positions (natural) = [3, 7, 11, 13, 14, 15, 19, 21, 22, 23, 25, 26, 27, 29, 30, 31]
Trial 0/200: bit_err=5, block_err=1
Trial 20/200: bit_err=168, block_err=21
Trial 40/200: bit_err=317, block_err=41
Trial 60/200: bit_err=452, block_err=61
Trial 80/200: bit_err=610, block_err=81
Trial 100/200: bit_err=741, block_err=101
Trial 120/200: bit_err=891, block_err=121
Trial 140/200: bit_err=1035, block_err=140
Trial 160/200: bit_err=1161, block_err=160
Trial 180/200: bit_err=1326, block_err=180
No-noise check: 0.4603125 0.995 mapping: br info_pos: [ 3  7 11 13 14 15 19 21 22 23 25 26 27 29 30 31]


In [4]:
instrumented_n4_check()

info_positions (natural): [1, 3]
u (natural): [0, 0, 0, 1]
x: [1, 1, 1, 1]
llr: [-27.63102111592755, -27.63102111592755, -27.63102111592755, -27.63102111592755]
root: n=4, llr_sub=[-27.63102111592755, -27.63102111592755, -27.63102111592755, -27.63102111592755], frozen_sub=[True, False, True, False]
 f: a=[-27.63102111592755, -27.63102111592755], b=[-27.63102111592755, -27.63102111592755] -> f=[26.937840546492907, 26.937840546492907]
  root-L: n=2, llr_sub=[26.937840546492907, 26.937840546492907], frozen_sub=[True, False]
   f: a=[26.937840546492907], b=[26.937840546492907] -> f=[26.244693365930964]
    root-L-L: n=1, llr_sub=[26.244693365930964], frozen_sub=[True]
     => leaf frozen -> 0
   uhat_up returned: [0]
   g: sign_term=[1], g_llr=[53.875681092985815]
    root-L-R: n=1, llr_sub=[53.875681092985815], frozen_sub=[False]
     => leaf decide -> 0
   uhat_low returned: [0]
   combined: [0, 0]
 uhat_up returned: [0, 0]
 g: sign_term=[1, 1], g_llr=[-55.2620422318551, -55.262042231855